In [1]:
import numpy as np
import pandas as pd

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True) 
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)

In [174]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [175]:
y_train = train.Survived
train.drop(['Survived',], axis=1, inplace=True)


In [176]:
train['is_test'] = False
test['is_test'] = True

In [177]:
df = pd.concat([train, test], axis=0)
df.index = range(len(df))
df.columns = df.columns.str.lower()
df.shape

(1309, 12)

In [178]:
df.head(10)

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,False
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,False
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,False
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,False
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,False


In [179]:
from collections import Counter

In [180]:
one_big_text = " ".join(df.name)
words = one_big_text.replace('/',' / ').split()
most_common = Counter(words).most_common()
most_common

In [184]:
from sklearn.tree import DecisionTreeClassifier

In [188]:
df['Mr. '] = df.name.str.contains('Mr. ').astype(int)
df['Miss.'] = df.name.str.contains('Miss.').astype(int)
df['Mrs.'] = df.name.str.contains('Mrs.').astype(int)
df['Master.'] = df.name.str.contains('Master.').astype(int)

In [189]:
df.head()

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,Mr.,Miss.,Mrs.,Master.
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,1,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False,0,0,1,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False,0,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False,0,0,1,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,1,0,0,0


In [190]:
df1 = df[(df['Mr. ']==1)]
df1.head()

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,Mr.,Miss.,Mrs.,Master.
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,False,1,0,0,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,False,1,0,0,0
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,False,1,0,0,0
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,False,1,0,0,0
12,13,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S,False,1,0,0,0


In [191]:
mr_age = df1.age.mean()
mr_age

32.228205128205126

In [192]:
df2 = df[(df['Miss.']==1)]
df2.isnull().sum()
miss_age = df2.age.mean()
miss_age

21.774238095238097

In [193]:
df3 = df[(df['Master.']==1)]
df3.isnull().sum()
master_age = df3.age.mean()
master_age

5.482641509433963

In [194]:
# пустые ячейки в возрасте заменим на средний возраст в зависимости от обращения, обращение Misses не рассматривается т.к. там
# все возраста заполнены
for i in range(1309):
    if df['Mr. '][i] == 1:
        df['age'][i] = mr_age
    if df['Miss.'][i] == 1:
        df['age'][i] = miss_age
    if df['Master.'][i] == 1:
        df['age'][i] = master_age
    if df['Mr. '][i] == 0 and df['Miss.'][i] == 0 and df['Master.'][i] == 0:
        df['age'][i] = df.age.mean() 
df.head()        

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,Mr.,Miss.,Mrs.,Master.
0,1,3,"Braund, Mr. Owen Harris",male,32.228205,1,0,A/5 21171,7.2500,NaN,S,False,1,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,29.890916,1,0,PC 17599,71.2833,C85,C,False,0,0,1,0
2,3,3,"Heikkinen, Miss. Laina",female,21.774238,0,0,STON/O2. 3101282,7.9250,NaN,S,False,0,1,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,29.879124,1,0,113803,53.1000,C123,S,False,0,0,1,0
4,5,3,"Allen, Mr. William Henry",male,32.228205,0,0,373450,8.0500,NaN,S,False,1,0,0,0


In [195]:
df.tail()

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_test,Mr.,Miss.,Mrs.,Master.
1304,1305,3,"Spector, Mr. Woolf",male,32.228205,0,0,A.5. 3236,8.0500,NaN,S,True,1,0,0,0
1305,1306,1,"Oliva y Ocana, Dona. Fermina",female,28.388691,0,0,PC 17758,108.9000,C105,C,True,0,0,0,0
1306,1307,3,"Saether, Mr. Simon Sivertsen",male,32.228205,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,True,1,0,0,0
1307,1308,3,"Ware, Mr. Frederick",male,32.228205,0,0,359309,8.0500,NaN,S,True,1,0,0,0
1308,1309,3,"Peter, Master. Michael J",male,5.482642,1,1,2668,22.3583,NaN,C,True,0,0,0,1


In [196]:
df["isMale"] = df.sex.replace({"male": 1, "female":0})
df.drop(["sex", "cabin", "ticket", "name", "passengerid"], axis=1, inplace=True)

In [197]:
df.head()

,pclass,age,sibsp,parch,fare,embarked,is_test,Mr.,Miss.,Mrs.,Master.,isMale
0,3,32.228205,1,0,7.2500,S,False,1,0,0,0,1
1,1,29.890916,1,0,71.2833,C,False,0,0,1,0,0
2,3,21.774238,0,0,7.9250,S,False,0,1,0,0,0
3,1,29.879124,1,0,53.1000,S,False,0,0,1,0,0
4,3,32.228205,0,0,8.0500,S,False,1,0,0,0,1


In [198]:
df_dummies = pd.get_dummies(df, columns=['pclass','embarked'])

In [199]:
df_dummies.head(10)

,age,sibsp,parch,fare,is_test,Mr.,Miss.,Mrs.,Master.,isMale,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
0,32.228205,1,0,7.2500,False,1,0,0,0,1,0,0,1,0,0,1
1,29.890916,1,0,71.2833,False,0,0,1,0,0,1,0,0,1,0,0
2,21.774238,0,0,7.9250,False,0,1,0,0,0,0,0,1,0,0,1
3,29.879124,1,0,53.1000,False,0,0,1,0,0,1,0,0,0,0,1
4,32.228205,0,0,8.0500,False,1,0,0,0,1,0,0,1,0,0,1
5,32.228205,0,0,8.4583,False,1,0,0,0,1,0,0,1,0,1,0
6,32.228205,0,0,51.8625,False,1,0,0,0,1,1,0,0,0,0,1
7,5.482642,3,1,21.0750,False,0,0,0,1,1,0,0,1,0,0,1
8,29.856361,0,2,11.1333,False,0,0,1,0,0,0,0,1,0,0,1
9,29.859089,1,0,30.0708,False,0,0,1,0,0,0,1,0,1,0,0


In [200]:
df_dummies.isnull().sum()

age           0
sibsp         0
parch         0
fare          1
is_test       0
Mr.           0
Miss.         0
Mrs.          0
Master.       0
isMale        0
pclass_1      0
pclass_2      0
pclass_3      0
embarked_C    0
embarked_Q    0
embarked_S    0
dtype: int64

In [201]:
df_dummies.fare.fillna(df_dummies.fare.mean(), inplace=True)

In [202]:
df_dummies.head()

,age,sibsp,parch,fare,is_test,Mr.,Miss.,Mrs.,Master.,isMale,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
0,32.228205,1,0,7.2500,False,1,0,0,0,1,0,0,1,0,0,1
1,29.890916,1,0,71.2833,False,0,0,1,0,0,1,0,0,1,0,0
2,21.774238,0,0,7.9250,False,0,1,0,0,0,0,0,1,0,0,1
3,29.879124,1,0,53.1000,False,0,0,1,0,0,1,0,0,0,0,1
4,32.228205,0,0,8.0500,False,1,0,0,0,1,0,0,1,0,0,1


In [203]:
X_train = df_dummies[df_dummies.is_test==False].drop('is_test', axis=1)
X_test = df_dummies[df_dummies.is_test==True].drop('is_test', axis=1)

In [204]:
X_test.head(10)

,age,sibsp,parch,fare,Mr.,Miss.,Mrs.,Master.,isMale,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
891,32.228205,0,0,7.8292,1,0,0,0,1,0,0,1,0,1,0
892,28.914992,1,0,7.0000,0,0,1,0,0,0,0,1,0,0,1
893,32.228205,0,0,9.6875,1,0,0,0,1,0,1,0,0,1,0
894,32.228205,0,0,8.6625,1,0,0,0,1,0,0,1,0,0,1
895,28.880136,1,1,12.2875,0,0,1,0,0,0,0,1,0,0,1
896,32.228205,0,0,9.2250,1,0,0,0,1,0,0,1,0,0,1
897,21.774238,0,0,7.6292,0,1,0,0,0,0,0,1,0,1,0
898,32.228205,1,1,29.0000,1,0,0,0,1,0,1,0,0,0,1
899,28.899033,0,0,7.2292,0,0,1,0,0,0,0,1,1,0,0
900,32.228205,2,0,24.1500,1,0,0,0,1,0,0,1,0,0,1


In [205]:
X_test.isnull().sum()

age           0
sibsp         0
parch         0
fare          0
Mr.           0
Miss.         0
Mrs.          0
Master.       0
isMale        0
pclass_1      0
pclass_2      0
pclass_3      0
embarked_C    0
embarked_Q    0
embarked_S    0
dtype: int64

In [206]:
columns = X_train.columns

In [207]:
from sklearn.preprocessing import StandardScaler

In [208]:
scaler = StandardScaler()

In [209]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [210]:
X_train_scaled = scaler.transform(X_train)
X_train_scaled_1 = pd.DataFrame(X_train_scaled, columns=columns )
X_train_scaled_1.head()

,age,sibsp,parch,fare,Mr.,Miss.,Mrs.,Master.,isMale,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
0,0.594152,0.432793,-0.473674,-0.502445,0.842717,-0.506655,-0.411450,-0.216803,0.737695,-0.565685,-0.510152,0.902587,-0.482043,-0.307562,0.619306
1,0.230286,0.432793,-0.473674,0.786845,-1.186637,-0.506655,2.430427,-0.216803,-1.355574,1.767767,-0.510152,-1.107926,2.074505,-0.307562,-1.614710
2,-1.033310,-0.474545,-0.473674,-0.488854,-1.186637,1.973729,-0.411450,-0.216803,-1.355574,-0.565685,-0.510152,0.902587,-0.482043,-0.307562,0.619306
3,0.228450,0.432793,-0.473674,0.420730,-1.186637,-0.506655,2.430427,-0.216803,-1.355574,1.767767,-0.510152,-1.107926,-0.482043,-0.307562,0.619306
4,0.594152,-0.474545,-0.473674,-0.486337,0.842717,-0.506655,-0.411450,-0.216803,0.737695,-0.565685,-0.510152,0.902587,-0.482043,-0.307562,0.619306


In [229]:
columns_t = X_test.columns
X_test_scaled = scaler.transform(X_test)
X_test_scaled_1 = pd.DataFrame(X_test_scaled, columns=columns_t )

In [230]:
from sklearn.model_selection import train_test_split

In [231]:
X_train_fin, X_val, y_train_fin, y_val = train_test_split(X_train_scaled_1, y_train, test_size=0.2)

In [232]:
X_train_fin.head()

,age,sibsp,parch,fare,Mr.,Miss.,Mrs.,Master.,isMale,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
100,-1.033310,-0.474545,-0.473674,-0.489442,-1.186637,1.973729,-0.411450,-0.216803,-1.355574,-0.565685,-0.510152,0.902587,-0.482043,-0.307562,0.619306
662,0.594152,-0.474545,-0.473674,-0.133225,0.842717,-0.506655,-0.411450,-0.216803,0.737695,1.767767,-0.510152,-1.107926,-0.482043,-0.307562,0.619306
600,0.104600,1.340132,0.767630,-0.104785,-1.186637,-0.506655,2.430427,-0.216803,-1.355574,-0.565685,1.960202,-1.107926,-0.482043,-0.307562,0.619306
417,-1.033310,-0.474545,2.008933,-0.386671,-1.186637,1.973729,-0.411450,-0.216803,-1.355574,-0.565685,1.960202,-1.107926,-0.482043,-0.307562,0.619306
444,0.594152,-0.474545,-0.473674,-0.485079,0.842717,-0.506655,-0.411450,-0.216803,0.737695,-0.565685,-0.510152,0.902587,-0.482043,-0.307562,0.619306


In [233]:
from sklearn.model_selection import GridSearchCV

In [336]:
parameter_candidates = {'max_depth': list(range(1,10)),'max_features': list(range(1,16)), 
                        'random_state' : list(range(1,10)), 'min_samples_split': (2,),
                        'min_samples_leaf': (1,2), 'criterion':('gini', 'entropy')}

In [337]:
gridsearch = GridSearchCV(DecisionTreeClassifier(), param_grid=parameter_candidates, scoring='accuracy', cv=7)

In [338]:
gridsearch.fit(X_train_fin, y_train_fin)

GridSearchCV(cv=7, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'random_state': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'min_samples_leaf': (1, 2), 'min_samples_split': (2,), 'criterion': ('gini', 'entropy'), 'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [339]:
sorted(gridsearch.grid_scores_, key = lambda x: -x.mean_validation_score)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.82444, std: 0.01834, params: {'max_depth': 5, 'random_state': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 8},
 mean: 0.82444, std: 0.03131, params: {'max_depth': 6, 'random_state': 9, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 5},
 mean: 0.82444, std: 0.02935, params: {'max_depth': 4, 'random_state': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 9},
 mean: 0.82444, std: 0.02935, params: {'max_depth': 4, 'random_state': 2, 'min_samples_leaf': 2, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 9},
 mean: 0.82303, std: 0.03484, params: {'max_depth': 5, 'random_state': 7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 6},
 mean: 0.82303, std: 0.03031, params: {'max_depth': 5, 'random_state': 8, 'min_samples_leaf': 2, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 8},
 mean: 0.82303, std: 0

In [340]:
gridsearch.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 8,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'random_state': 2}

In [341]:
max_depth = gridsearch.best_params_["max_depth"]
max_features = gridsearch.best_params_["max_features"]
min_samples_leaf = gridsearch.best_params_["min_samples_leaf"]
min_samples_split = gridsearch.best_params_["min_samples_split"]
random_state = gridsearch.best_params_["random_state"]
criterion = gridsearch.best_params_["criterion"]

In [342]:
gridsearch.best_score_

0.824438202247191

In [343]:
clf_final = DecisionTreeClassifier(max_depth = max_depth, max_features = max_features, min_samples_leaf = min_samples_leaf,
                                   min_samples_split = min_samples_split, random_state = random_state)

In [344]:
clf_final.fit(X_train_fin, y_train_fin)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=2, splitter='best')

In [356]:
# наиболее важными признаками оказались: половая принадлежность, стоимость билета, нахождение в 3 классе
clf_final.feature_importances_

array([ 0.01428222,  0.07717336,  0.01120337,  0.14362682,  0.00068387,
        0.        ,  0.        ,  0.05589104,  0.54272735,  0.        ,
        0.01382437,  0.1359285 ,  0.00465911,  0.        ,  0.        ])

In [345]:
y_val_pred = clf_final.predict(X_val)

In [346]:
from sklearn.metrics import accuracy_score

In [347]:
accuracy_score(y_val, y_val_pred)

0.87150837988826813

In [348]:
clf_final.fit(X_train_scaled_1, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=2, splitter='best')

In [349]:
clf_final.predict_proba(X_test_scaled_1)[:10]

array([[ 0.90751445,  0.09248555],
       [ 0.31578947,  0.68421053],
       [ 0.90751445,  0.09248555],
       [ 0.90751445,  0.09248555],
       [ 0.6       ,  0.4       ],
       [ 0.90751445,  0.09248555],
       [ 0.31578947,  0.68421053],
       [ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.90751445,  0.09248555]])

In [350]:
predictions = clf_final.predict(X_test_scaled_1)

In [351]:
submussion = 'PassengerId,Survived\n'
submussion += "\n".join(["{},{}".format(pid, prediction) for pid, prediction in zip(test.PassengerId, predictions)])

In [352]:
with open('submission.txt', 'w') as file:
    file.write(submussion)

In [353]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [354]:
y_train_1 = pd.get_dummies(y_train, columns=['Survived'])
list(y_train_1.columns)

[0, 1]

In [283]:
get_tree_dot_view(clf_final, list(X_train.columns), ['0','1'])

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="isMale <= -0.309\ngini = 0.473\nsamples = 891\nvalue = [549, 342]\nclass = 0", fillcolor="#e5813960"] ;
1 [label="pclass_3 <= -0.103\ngini = 0.383\nsamples = 314\nvalue = [81, 233]\nclass = 1", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="fare <= -0.067\ngini = 0.1\nsamples = 170\nvalue = [9, 161]\nclass = 1", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="parch <= 1.388\ngini = 0.18\nsamples = 70\nvalue = [7, 63]\nclass = 1", fillcolor="#399de5e3"] ;
2 -> 3 ;
4 [label="fare <= -0.08\ngini = 0.206\nsamples = 60\nvalue = [7, 53]\nclass = 1", fillcolor="#399de5dd"] ;
3 -> 4 ;
5 [label="gini = 0.183\nsamples = 59\nvalue = [6, 53]\nclass = 1", fillcolor="#399de5e2"] ;
4 -> 5 ;
6 [label="gini = 0.0\nsamples = 1\nvalue = [1, 0]\nclass = 0", fillcolor="#e58139ff"] ;
4 -> 6 ;
7 [label="gini = 0.0\nsamples = 10\nvalue = [0, 10]\nclass = 1", fillcolor="#399de5ff"] ;
3 -> 7 